# 1. Intro
This example is powered by [https://www.kaggle.com/code/amyjang/tensorflow-mnist-cnn-tutorial](https://www.kaggle.com/code/amyjang/tensorflow-mnist-cnn-tutorial)\
This example is an introduction to Convolutional Neural Networks using TensorFlow 2.x Keras API.\
The dataset for training and validation is the [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset, a dataset of handwritten digits 0-9, and here I will use a Sequential CNN to guess which digit was drawn.\
This ***model*** setup reaches 99.3% accuracy.

Import all mandatory packages ***?and change the accelerator from None to GPU?***.

In [ ]:
import tensorflow as tf
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib as mpltl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

print(f"tensorflow:{tf.__version__}")
print(f"seaborn:{sns.__version__}")
print(f"numpy:{np.__version__}")
print(f"pandas:{pd.__version__}")
print(f"matplotlib:{mpltl.__version__}")

# 2. Data Preprocessing
Before building any ML model, it is important to preprocess the data.\
In fact, data preprocessing will generally take up the most time in any ML pipeline.\
The following module goes over the steps to preprocess the MNIST dataset for our purposes.

## 2.1 Load Data
The first step is to load the data and divide it into a training and testing dataset.\
The MNIST dataset can be downloaded directly from TensorFlow and has already been divided.

* x_train is the dataset of 28x28 images of handwritten digits that the model will be trained on
* y_train is the dataset of labels that correspond to x_train.
* x_test is the dataset of 28x28 images of handwritten digits that the model will be tested on
* y_test is the dataset of labels that correspond to x_test

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Let see the counts of each digit present in the training dataset.

In [ ]:
sns.countplot(y_train)

There are similar counts for each digit.
This is good as the ***model*** will have enough images for each class to train the features for each class.
There is no need to downsample or upweigh.

PTFIXME: 1 pic for every number? WTF ?

## 2.2 Check for NaN Values
There shell be no NaN values in the dataset.
There is no need to preprocess the data to deal with Nan's.

In [ ]:
assert False == np.isnan(x_train).any()

In [ ]:
assert False == np.isnan(x_test).any()

## 2.3 Normalization and Reshaping
Since the values in the x_train dataset are 28x28 images, the input shape must be specified so that the model will know what is being inputed.\
The first convolution layer expects a single 60000x28x28x1 tensor instead of 60000 28x28x1 tensors.

Models generally run better on normalized values.\
The best way to normalize the data depends on each individual dataset.\
For the MNIST dataset, I want each value to be between 0.0 and 1.0.\
As all values originally fall under the 0.0-255.0 range, divide by 255.0.

In [ ]:
input_shape = (28, 28, 1)

x_train=x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_train=x_train / 255.0
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 1)
x_test=x_test/255.0

## 2.4 Label Encoding
The labels for the training and the testing dataset are currently categorical and is not continuous.
To include categorical dataset in the ***model***, the labels should be converted to one-hot encodings.

For example, 2 becomes [0,0,1,0,0,0,0,0,0,0] and 7 becomes [0,0,0,0,0,0,0,1,0,0].

Transforming the labels into one-hot encodings here.

In [ ]:
y_train = tf.one_hot(y_train.astype(np.int32), depth=10)
y_test = tf.one_hot(y_test.astype(np.int32), depth=10)

## 2.5 Visualize Data
Visualizing an image in the dataset.
The image is an image of a handwritten 5.
The one-hot encoding holds the value of 5.

In [ ]:
plt.imshow(x_train[100][:,:,0])
print(y_train[100])

# 3. CNN
Time to build the CNN model.

## 3.1 Define the model
Have to define now
* batch_size
* num_classes,
* and epochs.

Try changing the values and test how different values affect the accuracy of the CNN model.

In [ ]:
batch_size = 64
num_classes = 10
epochs = 5

Building the ***model***.\
The ***model*** contains various layers stacked on top of each other.\
The output of one layer feeds into the input of the next layer.

Conv2D layers are convolutions.\
Each filter (32 in the first two convolution layers and 64 in the next two convolution layers) transforms a part of the image (5x5 for the first two Conv2D layers and 3x3 for the next two Conv2D layers).\
The transformation is applied on the whole image.

***MaxPool2D*** is a downsampling filter.\
It reduces a 2x2 matrix of the image to a single pixel with the maximum value of the 2x2 matrix.\
The filter aims to conserve the main features of the image while reducing the size.

***Dropout*** is a regularization layer.\
In this ***model***, 25% of the nodes in the layer are randomly ignores, allowing the network to learn different features.\
This prevents overfitting.

***relu*** is the rectifier, and it is used to find nonlinearity in the data.\
It works by returning the input value if the input value >= 0.\
If the input is negative, it returns 0.

***Flatten*** converts the tensors into a 1D vector.

The ***Dense*** layers are an artificial neural network (ANN).\
The last layer returns the probability that an image is in each class (one for each digit).

As this ***model*** aims to categorize the images, I will use a categorical_crossentropy loss function.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu', input_shape=input_shape),
    tf.keras.layers.Conv2D(32, (5,5), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(strides=(2,2)),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.RMSprop(epsilon=1e-08), loss='categorical_crossentropy', metrics=['acc'])

## 3.2 Fit the Training Data
The next step is to fit the training data.\
If I achieve a certain level of accuracy, it may not be necessary to continue training the model, especially if time and resources are limited.

Lets define a ***DoneCallBack*** so that if 99.5% accuracy is achieved, the ***model*** stops training.\
The ***model*** is not likely to stop prematurely if only 5 epochs are specified.\
Try it out with more epochs.


In [ ]:
class DoneCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.995):
      print("\nReached 99.5% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = DoneCallback()

Testing the ***model*** on a validation dataset prevents overfitting of the data.\
Lets' specify a 10% validation and 90% training split.

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_split=0.1,
                    callbacks=[callbacks])

# 4. Evaluate the model

## 4.1 Loss and Accuracy Curves
Let's evaluate the loss and accuracy of the ***model***.\
The accuracy increases over time and the loss decreases over time.\
However, the accuracy of the validation set seems to slightly decrease towards the end even thought our training accuracy increased.\
Running the ***model*** for more epochs might cause the ***model*** to be susceptible to overfitting.

In [ ]:
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training Loss")
#ax[0].plot(history.history['val_loss'], color='r', label="Validation Loss",axes =ax[0])
ax[0].plot(history.history['val_loss'], color='r', label="Validation Loss")
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training Accuracy")
ax[1].plot(history.history['val_acc'], color='r',label="Validation Accuracy")
legend = ax[1].legend(loc='best', shadow=True)

The accuracy increases over time and the loss decreases over time.\
However, the accuracy of the validation set seems to slightly decrease towards the end even thought the Training Accuracy increased.\
Running the ***model*** for more epochs might cause the ***model*** to be susceptible to overfitting.

## 4.2 Predict Results
The ***model*** runs pretty well, with an accuracy around 99.3% on the testing data.

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

## 4.3 Confusion Matrix
Let's compute and plot a Confusion Matrix using TensorFlow.

In [ ]:
# Predict the values from the testing dataset
Y_pred = model.predict(x_test)
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_pred,axis = 1) 
# Convert testing observations to one hot vectors
Y_true = np.argmax(y_test,axis = 1)
# compute the confusion matrix
confusion_mtx = tf.math.confusion_matrix(Y_true, Y_pred_classes) 

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx, annot=True, fmt='g')

Crossing X and Y axises here contains a number, the higher the number(error) - the higher confusion between these 2 numbers, i. e. probabily of error.

Let's agree, by saying ***{1,9}*** I mean a cross of X equal to 1 and Y equal to 9 axises.

Here seems to be a slightly higher confusion between ***{2,7}*** which is ***7*** and ***{3,5}*** which is ***7*** again.

This is reasonable:
* 2's and 7's might look similar if 2's tail is missing
* 3's and 5's can be mistaken when the top side is unclear written

As opposite ***{1,9}*** is ***0*** - yeah, 1 and 9 are hardly confusable.